In [1]:
import pandas as pd
import numpy as np
import itertools

import FinanceDataReader as fdr
from scipy.stats import gmean

import warnings
warnings.filterwarnings("ignore")

In [70]:
kospi_monthly_df = pd.read_csv('./datasets/kospi_monthly.csv') # kospi 지수 월데이터 17.01~
index_monthly_df = pd.read_csv('./datasets/monthly_data.csv') # kospi 모든주 월데이터 17.01~
index_df = pd.read_csv('./datasets/ks_ohlcv.csv', encoding='cp949') # kospi 모든주 일데이터 17.01.02~
fm = pd.read_csv('./datasets/ks_fm.csv')

index_monthly_df.drop('Unnamed: 0', axis=1, inplace=True)
kospi_monthly_df.drop('Unnamed: 0', axis=1, inplace=True)


In [71]:
# kospi 월별 수익률
kospi_monthly_df['Change'] = (kospi_monthly_df['Close'] - kospi_monthly_df['Close'].shift(1))/kospi_monthly_df['Close'].shift(1)

In [72]:
np.array(kospi_monthly_df.iloc[-60:]['Date'].values)

array(['2017-05-31', '2017-06-30', '2017-07-31', '2017-08-31',
       '2017-09-29', '2017-10-31', '2017-11-30', '2017-12-28',
       '2018-01-31', '2018-02-28', '2018-03-30', '2018-04-30',
       '2018-05-31', '2018-06-29', '2018-07-31', '2018-08-31',
       '2018-09-28', '2018-10-31', '2018-11-30', '2018-12-28',
       '2019-01-31', '2019-02-28', '2019-03-29', '2019-04-30',
       '2019-05-31', '2019-06-28', '2019-07-31', '2019-08-30',
       '2019-09-30', '2019-10-31', '2019-11-29', '2019-12-30',
       '2020-01-31', '2020-02-28', '2020-03-31', '2020-04-29',
       '2020-05-29', '2020-06-30', '2020-07-31', '2020-08-31',
       '2020-09-29', '2020-10-30', '2020-11-30', '2020-12-30',
       '2021-01-29', '2021-02-26', '2021-03-31', '2021-04-30',
       '2021-05-31', '2021-06-30', '2021-07-30', '2021-08-31',
       '2021-09-30', '2021-10-29', '2021-11-30', '2021-12-30',
       '2022-01-28', '2022-02-28', '2022-03-31', '2022-04-29'],
      dtype=object)

In [73]:
kospi_var = pd.DataFrame(kospi_monthly_df.iloc[-60:][['year','month','Change']])

In [74]:
var = pd.DataFrame()
for i in range(len(index_monthly_df['회사코드'].unique())):
    data = index_monthly_df[index_monthly_df['회사코드'] == index_monthly_df['회사코드'].unique()[i]]
    aa = ((data['종가'] - data['종가'].shift(1))/data['종가'].shift(1))
    var[f"{index_monthly_df['회사코드'].unique()[i]}"] = aa.values

In [75]:
bb = []
for i in range(len(index_monthly_df['회사코드'].unique())):
    bb.append(kospi_var['Change'].cov(var.iloc[-60:][f"{index_monthly_df['회사코드'].unique()[i]}"])/kospi_var['Change'].var())

In [76]:
# 60beta 데이터 프레임 만들기
c = index_monthly_df['회사코드'].unique()
beta = pd.DataFrame(c)
beta.columns = ['회사코드']
beta['60beta'] = bb

In [77]:
# 60beta 합쳐주기
index_monthly_df = pd.merge(index_monthly_df, beta, left_on='회사코드', right_on='회사코드', how='inner')

In [78]:
index_monthly_df['월별수익률'] = (index_monthly_df['종가'] - index_monthly_df['종가'].shift(1))/index_monthly_df['종가'].shift(1)

In [79]:
gm = []
for i in range(len(index_monthly_df['회사코드'].unique())):
    vf = index_monthly_df[index_monthly_df['회사코드'] == index_monthly_df['회사코드'].unique()[i]].iloc[-60:]['월별수익률']
    vf=np.array(vf) +1
    gm.append(gmean(vf))

In [80]:
# index_monthly_df.to_csv('./datasets/index_monthly_all.csv')

In [81]:
# 스타일 배분 준비
style_index_df = index_monthly_df[(index_monthly_df['날짜'] == '2022-04-29')]
style_index_df.drop('월별수익률',axis=1,inplace=True)

In [82]:
# 60개월평균수익률 삽입
style_index_df['60개월평균수익률'] = gm

In [83]:
# 시총 기준으로 정렬해주기
style_index_df.sort_values('시가총액', ascending=False, inplace=True)

In [84]:
list66 = [  3070,  11500,   7690,    120,   4370,   1530,  17670, 181710,
        66570,  34310,   9180,   4450,  36530,   8110,   3550, 117580,
         2810,  15360,   6400,   1880,  37270,  35420,   1940,   9160,
           80, 100840,  92220, 229640,   2460,   8490,  75580,  16800,
         3780,   5380,   1790,   6880,  17390,    660,  12750,  52690,
          480, 155900,   9410,  11790,   1720,  34120,   3720,   1750,
        17960,   6200,   4080,   4700,   2710,  24900,  24090,  93370,
        34590,    270,  42700,   2840,    990,   5930,    100,  88980,
       195870,  23450]

In [85]:
# ff = []
# for i in range(len(list66)):
#     aa = style_index_df[style_index_df['회사코드'] == list66[i]]
#     ff.append(aa)
# style_index66_df = pd.concat(ff)

In [86]:
# style_index66_df.sort_values('기대수익률', ascending=False)

In [87]:
# style_index_df['기대수익률'] = 0.15 + ((style_index_df['60개월평균수익률']-1)*100 - 0.2)*style_index_df['60beta']

In [88]:
# style_index_df.sort_values('기대수익률', ascending=False)

In [89]:
# 회사코드 6개자리로 만들기
aa = style_index_df['회사코드']
bb = []
for i in range(len(aa)):
    bb.append(format(aa.values[i], '06'))
style_index_df['회사코드'] = bb

In [90]:
# 월말 주가와 기준치가 되는 값들 병합해주기
style_index_df = pd.merge(style_index_df, fm , left_on='회사코드', right_on='티커', how = 'inner')
style_index_df.drop('티커', axis=1, inplace=True)
style_index_df['회사코드']=style_index_df['회사코드'].astype('int64')

In [91]:
# style_index_df.to_csv('./datasets/style_index_2204.csv')

In [92]:
style_index_df['기대수익률'] = 0.15 + ((style_index_df['60개월평균수익률']-1)*100 - 0.2)*style_index_df['60beta']

In [93]:
# list66 = [  3070,  11500,   7690,    120,   4370,   1530,  17670, 181710,
#         66570,  34310,   9180,   4450,  36530,   8110,   3550, 117580,
#          2810,  15360,   6400,   1880,  37270,  35420,   1940,   9160,
#            80, 100840,  92220, 229640,   2460,   8490,  75580,  16800,
#          3780,   5380,   1790,   6880,  17390,    660,  12750,  52690,
#           480, 155900,   9410,  11790,   1720,  34120,   3720,   1750,
#         17960,   6200,   4080,   4700,   2710,  24900,  24090,  93370,
#         34590,    270,  42700,   2840,    990,   5930,    100,  88980,
#        195870,  23450]

In [94]:
new_style = pd.read_csv('./datasets/new_style.csv')

In [95]:
# bv = np.array(new_style['대형가치주코드'][0:5].astype(int))
# bg = np.array(new_style['대형성장주코드'][0:11].astype(int))
# msg = np.array(new_style['중소형성장주코드'].astype(int))
# msv = np.array(new_style['중소형가치주코드'][0:24].astype(int))

bv = list(new_style['대형가치주코드'][0:5].astype(int))
bg = list(new_style['대형성장주코드'][0:11].astype(int))
msg = list(new_style['중소형성장주코드'].astype(int))
msv = list(new_style['중소형가치주코드'][0:24].astype(int))

list66 = bv + bg + msg + msv
len(list66)

66

In [96]:
decide_new = pd.DataFrame(list66)
decide_new.columns = ['회사코드']

blank = []
for i in range(len(bv)):
    blank.append('대형가치주')
for i in range(len(bg)):
    blank.append('대형성장주')
for i in range(len(msg)):
    blank.append('중소형성장주')
for i in range(len(msv)):
    blank.append('중소형가치주')

decide_new['분류기준'] = blank

In [97]:
big = [ 5930,   660, 35420,  6400,  5380,   270, 88980, 11790,   100,  990, 12750,    80]
ms = [ 93370,  42700, 195870,  17390,  34120,   2840,  17960,   1720,
         9160,  92220,   8490,   1790,  16800,  75580,   9410,    480,
         4700,  24090,   2460,   1940,   2710, 229640,  15360,  23450,
         1750,   4080, 100840,   6880,  34590,   3720,   3780,  24900,
         6200, 155900]
growth=[  9160,  24090,   6200,   2710,   6880, 195870,   2460,  34120,
        93370,  92220,  52690,    990,   3780,  37270,   3720,  11790,
         4080,  17390,     80,  24900,  23450,    660,  75580,   6400,
        17960,    270,  35420,    100,   2840,   1720]
value=[ 42700,   1790,  34590,   5380,   8490, 100840,  16800,   4700,
          480,   9410,   1750,  15360,   1940]
list66 = big + ms + growth + value
len(list66)

89

In [98]:
decide = pd.DataFrame(list66)
decide.columns = ['회사코드']
decide_uni = pd.DataFrame(decide['회사코드'].unique())
decide_uni.columns = ['회사코드']

blank = []
for i in range(len(big)):
    blank.append('대형주')
for i in range(len(ms)):
    blank.append('중소형주')
for i in range(len(growth)):
    blank.append('성장주')
for i in range(len(value)):
    blank.append('가치주')

decide['분류기준'] = blank

blank = []
gj = ''
for i in range(len(decide['회사코드'].unique())):
    for j in range(len(decide[decide['회사코드'] == decide['회사코드'].unique()[i]])):
        gj = gj + decide[decide['회사코드'] == decide['회사코드'].unique()[i]].reset_index()['분류기준'][j]
    blank.append(gj)
    gj = ''

decide_uni['분류기준'] = blank

In [99]:
# 66개 주식으로 필터링해주기
ff = []
for i in range(len(decide_new)):
    aa = style_index_df[style_index_df['회사코드'] == decide_new['회사코드'][i]]
    ff.append(aa)
style_index66_df = pd.concat(ff)

In [100]:
# 대형주/중소형주 구분 : 시총 100위기준으로
ls_df = style_index_df.head(100)
ss_df = style_index_df.drop(index = ls_df.index)

In [101]:
# 가치주/성장주로 구분 : 가치주는 [PER > 2.5 & PER < 10]
vs_df = style_index_df[(style_index_df['PER'] > 2.5) & (style_index_df['PER'] < 10) & (style_index_df['PBR'] < 1)]
gs_df = style_index_df.drop(index = vs_df.index)

In [102]:
lsvs_df = pd.merge(ls_df,vs_df, how='inner').sort_values(['기대수익률'],ascending=False)
lsgs_df = pd.merge(ls_df,gs_df, how='inner').sort_values(['기대수익률'],ascending=False)
ssvs_df = pd.merge(ss_df,vs_df, how='inner').sort_values(['기대수익률'],ascending=False)
ssgs_df = pd.merge(ss_df,gs_df, how='inner').sort_values(['기대수익률'],ascending=False)

In [103]:
section = [lsvs_df, lsgs_df, ssvs_df, ssgs_df]
section_list = ["lsvs66_df", "lsgs66_df", "ssvs66_df", "ssgs66_df"]

for j,df in enumerate(section_list):
    ff = []
    for i in range(len(style_index66_df)):
        gg = section[j][section[j]['회사코드'] == style_index66_df['회사코드'].values[i]]
        ff.append(gg)
    globals()[f'{df}'] = pd.concat(ff)
    globals()[f'{df}'].sort_values('기대수익률', ascending=False, inplace=True)

In [104]:
for j,df in enumerate(section_list):
    print(df)

lsvs66_df
lsgs66_df
ssvs66_df
ssgs66_df


In [105]:
ff = [] 
for i in range(len(style_index66_df)):
    gg = lsvs_df[lsvs_df['회사코드'] == style_index66_df['회사코드'].values[i]]
    ff.append(gg)
lsvs66_df = pd.concat(ff)
lsvs66_df.sort_values('기대수익률', ascending=False, inplace=True)

ff = [] 
for i in range(len(style_index66_df)):
    gg = lsgs_df[lsgs_df['회사코드'] == style_index66_df['회사코드'].values[i]]
    ff.append(gg)
lsgs66_df = pd.concat(ff)
lsgs66_df.sort_values('기대수익률', ascending=False, inplace=True)

ff = [] 
for i in range(len(style_index66_df)):
    gg = ssvs_df[ssvs_df['회사코드'] == style_index66_df['회사코드'].values[i]]
    ff.append(gg)
ssvs66_df = pd.concat(ff)
ssvs66_df.sort_values('기대수익률', ascending=False, inplace=True)

ff = [] 
for i in range(len(style_index66_df)):
    gg = ssgs_df[ssgs_df['회사코드'] == style_index66_df['회사코드'].values[i]]
    ff.append(gg)
ssgs66_df = pd.concat(ff)
ssgs66_df.sort_values('기대수익률', ascending=False, inplace=True)

In [106]:
# result = list(itertools.combinations((ssgs66_df['회사코드'].values),17))
# print("**경우의 수 : %s개" % len(result))
# print(result)

In [107]:
len(ssgs66_df)

36

In [108]:
style_index66_df.to_csv('./samples.csv')

In [109]:
dfdf = pd.read_csv('./samples.csv',).drop('Unnamed: 0', axis=1)
dfdf

,날짜,시가,고가,저가,종가,거래량,회사코드,시가총액,60beta,60개월평균수익률,BPS,PER,PBR,EPS,DIV,DPS,기대수익률
0,2022-04-29,187000,188000,184000,186000,767005,5380,39742282782000,1.202404,1.004275,289609,9.85,0.65,18979,2.67,5000,0.423507
1,2022-04-29,56800,57300,56700,57000,640368,17670,13020672034800,0.670489,1.005375,53218,7.93,1.07,7191,11.68,6660,0.376312
2,2022-04-29,72600,73400,72500,73300,252247,3550,14003834694000,0.943834,1.002301,132574,4.76,0.56,15504,3.79,2800,0.178381
3,2022-04-29,33400,33500,33150,33450,83152,29780,3934255035200,0.402618,0.997066,70408,6.47,0.47,5164,6.89,2300,-0.048669
4,2022-04-29,121000,123500,121000,123000,27947,120,2794125567500,0.377914,0.994966,178766,66.54,0.69,1841,0.00,0,-0.115830
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,2022-04-29,13600,13750,13550,13700,13209,1750,261602505000,0.978901,1.010409,32750,2.20,0.42,6206,6.59,900,0.973198
62,2022-04-29,17750,20250,17750,18300,372841,100840,137372811300,0.796373,1.000384,37171,11.22,0.59,1957,3.64,800,0.021326
63,2022-04-29,27500,28500,27100,27900,46978,34590,122055636600,0.195433,0.998626,46433,31.08,0.60,901,4.46,1250,0.084053
64,2022-04-29,44950,45500,44500,44800,918,4450,91628203779,0.280541,0.999451,78782,57.44,0.57,780,2.79,1250,0.078497


In [110]:
stocklist = pd.read_csv('./datasets/stocklist.csv')
stcl = stocklist[['Name', 'Symbol']]
stcl

,Name,Symbol
0,BNK금융지주,138930
1,DB하이텍,990
2,HJ중공업,97230
3,JB금융지주,175330
4,KB금융,105560
...,...,...
790,효성첨단소재,298050
791,효성티앤씨,298020
792,효성화학,298000
793,흥국화재,540


In [111]:
style_index66_df = pd.merge(style_index66_df, stcl, left_on='회사코드', right_on='Symbol', how='inner')
style_index66_df

,날짜,시가,고가,저가,종가,거래량,회사코드,시가총액,60beta,60개월평균수익률,BPS,PER,PBR,EPS,DIV,DPS,기대수익률,Name,Symbol
0,2022-04-29,187000,188000,184000,186000,767005,5380,39742282782000,1.202404,1.004275,289609,9.85,0.65,18979,2.67,5000,0.423507,현대차,5380
1,2022-04-29,56800,57300,56700,57000,640368,17670,13020672034800,0.670489,1.005375,53218,7.93,1.07,7191,11.68,6660,0.376312,SK텔레콤,17670
2,2022-04-29,72600,73400,72500,73300,252247,3550,14003834694000,0.943834,1.002301,132574,4.76,0.56,15504,3.79,2800,0.178381,LG,3550
3,2022-04-29,33400,33500,33150,33450,83152,29780,3934255035200,0.402618,0.997066,70408,6.47,0.47,5164,6.89,2300,-0.048669,삼성카드,29780
4,2022-04-29,121000,123500,121000,123000,27947,120,2794125567500,0.377914,0.994966,178766,66.54,0.69,1841,0.00,0,-0.115830,CJ대한통운,120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,2022-04-29,13600,13750,13550,13700,13209,1750,261602505000,0.978901,1.010409,32750,2.20,0.42,6206,6.59,900,0.973198,한양증권,1750
62,2022-04-29,17750,20250,17750,18300,372841,100840,137372811300,0.796373,1.000384,37171,11.22,0.59,1957,3.64,800,0.021326,SNT에너지,100840
63,2022-04-29,27500,28500,27100,27900,46978,34590,122055636600,0.195433,0.998626,46433,31.08,0.60,901,4.46,1250,0.084053,인천도시가스,34590
64,2022-04-29,44950,45500,44500,44800,918,4450,91628203779,0.280541,0.999451,78782,57.44,0.57,780,2.79,1250,0.078497,삼화왕관,4450


In [112]:
style_index66_df.to_csv('./sample.csv')

In [113]:
style_index66_df = pd.merge(style_index66_df, decide_new, left_on='회사코드', right_on='회사코드', how='inner')
style_index66_df

,날짜,시가,고가,저가,종가,거래량,회사코드,시가총액,60beta,60개월평균수익률,BPS,PER,PBR,EPS,DIV,DPS,기대수익률,Name,Symbol,분류기준
0,2022-04-29,187000,188000,184000,186000,767005,5380,39742282782000,1.202404,1.004275,289609,9.85,0.65,18979,2.67,5000,0.423507,현대차,5380,대형가치주
1,2022-04-29,56800,57300,56700,57000,640368,17670,13020672034800,0.670489,1.005375,53218,7.93,1.07,7191,11.68,6660,0.376312,SK텔레콤,17670,대형가치주
2,2022-04-29,72600,73400,72500,73300,252247,3550,14003834694000,0.943834,1.002301,132574,4.76,0.56,15504,3.79,2800,0.178381,LG,3550,대형가치주
3,2022-04-29,33400,33500,33150,33450,83152,29780,3934255035200,0.402618,0.997066,70408,6.47,0.47,5164,6.89,2300,-0.048669,삼성카드,29780,대형가치주
4,2022-04-29,121000,123500,121000,123000,27947,120,2794125567500,0.377914,0.994966,178766,66.54,0.69,1841,0.00,0,-0.115830,CJ대한통운,120,대형가치주
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,2022-04-29,13600,13750,13550,13700,13209,1750,261602505000,0.978901,1.010409,32750,2.20,0.42,6206,6.59,900,0.973198,한양증권,1750,중소형가치주
62,2022-04-29,17750,20250,17750,18300,372841,100840,137372811300,0.796373,1.000384,37171,11.22,0.59,1957,3.64,800,0.021326,SNT에너지,100840,중소형가치주
63,2022-04-29,27500,28500,27100,27900,46978,34590,122055636600,0.195433,0.998626,46433,31.08,0.60,901,4.46,1250,0.084053,인천도시가스,34590,중소형가치주
64,2022-04-29,44950,45500,44500,44800,918,4450,91628203779,0.280541,0.999451,78782,57.44,0.57,780,2.79,1250,0.078497,삼화왕관,4450,중소형가치주


In [114]:
style_index66_df.to_csv('./sample.csv')

In [115]:
for i in range(len(style_index66_df)):
    print(style_index66_df['PER'][i])

9.85
7.93
4.76
6.47
66.54
7.9
2.53
34.93
7.11
24.78
40.71
6.41
65.17
9.17
170.14
36.32
78.07
16.79
56.63
86.49
6.56
14.56
38.92
2.58
5.5
40.25
20.2
41.15
24.85
15.22
4.93
9.02
9.94
12.5
18.65
11.64
18.26
5.04
31.18
21.49
16.99
3.4
14.19
10.67
14.91
7.22
10.79
10.95
3.45
3.84
2.6
8.1
24.36
7.44
5.15
10.5
15.62
2.28
3.33
6.25
7.4
2.2
11.22
31.08
57.44
20.33


In [116]:
len(style_index66_df)

66

In [117]:
style_index66_df

,날짜,시가,고가,저가,종가,거래량,회사코드,시가총액,60beta,60개월평균수익률,BPS,PER,PBR,EPS,DIV,DPS,기대수익률,Name,Symbol,분류기준
0,2022-04-29,187000,188000,184000,186000,767005,5380,39742282782000,1.202404,1.004275,289609,9.85,0.65,18979,2.67,5000,0.423507,현대차,5380,대형가치주
1,2022-04-29,56800,57300,56700,57000,640368,17670,13020672034800,0.670489,1.005375,53218,7.93,1.07,7191,11.68,6660,0.376312,SK텔레콤,17670,대형가치주
2,2022-04-29,72600,73400,72500,73300,252247,3550,14003834694000,0.943834,1.002301,132574,4.76,0.56,15504,3.79,2800,0.178381,LG,3550,대형가치주
3,2022-04-29,33400,33500,33150,33450,83152,29780,3934255035200,0.402618,0.997066,70408,6.47,0.47,5164,6.89,2300,-0.048669,삼성카드,29780,대형가치주
4,2022-04-29,121000,123500,121000,123000,27947,120,2794125567500,0.377914,0.994966,178766,66.54,0.69,1841,0.00,0,-0.115830,CJ대한통운,120,대형가치주
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,2022-04-29,13600,13750,13550,13700,13209,1750,261602505000,0.978901,1.010409,32750,2.20,0.42,6206,6.59,900,0.973198,한양증권,1750,중소형가치주
62,2022-04-29,17750,20250,17750,18300,372841,100840,137372811300,0.796373,1.000384,37171,11.22,0.59,1957,3.64,800,0.021326,SNT에너지,100840,중소형가치주
63,2022-04-29,27500,28500,27100,27900,46978,34590,122055636600,0.195433,0.998626,46433,31.08,0.60,901,4.46,1250,0.084053,인천도시가스,34590,중소형가치주
64,2022-04-29,44950,45500,44500,44800,918,4450,91628203779,0.280541,0.999451,78782,57.44,0.57,780,2.79,1250,0.078497,삼화왕관,4450,중소형가치주


In [118]:
# dddd = []
# for i in range(len(style_index66_df)):    
#     dddd.append(format(style_index66_df['회사코드'][i], '06'))
# dddd

In [119]:
style_index66_df

,날짜,시가,고가,저가,종가,거래량,회사코드,시가총액,60beta,60개월평균수익률,BPS,PER,PBR,EPS,DIV,DPS,기대수익률,Name,Symbol,분류기준
0,2022-04-29,187000,188000,184000,186000,767005,5380,39742282782000,1.202404,1.004275,289609,9.85,0.65,18979,2.67,5000,0.423507,현대차,5380,대형가치주
1,2022-04-29,56800,57300,56700,57000,640368,17670,13020672034800,0.670489,1.005375,53218,7.93,1.07,7191,11.68,6660,0.376312,SK텔레콤,17670,대형가치주
2,2022-04-29,72600,73400,72500,73300,252247,3550,14003834694000,0.943834,1.002301,132574,4.76,0.56,15504,3.79,2800,0.178381,LG,3550,대형가치주
3,2022-04-29,33400,33500,33150,33450,83152,29780,3934255035200,0.402618,0.997066,70408,6.47,0.47,5164,6.89,2300,-0.048669,삼성카드,29780,대형가치주
4,2022-04-29,121000,123500,121000,123000,27947,120,2794125567500,0.377914,0.994966,178766,66.54,0.69,1841,0.00,0,-0.115830,CJ대한통운,120,대형가치주
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,2022-04-29,13600,13750,13550,13700,13209,1750,261602505000,0.978901,1.010409,32750,2.20,0.42,6206,6.59,900,0.973198,한양증권,1750,중소형가치주
62,2022-04-29,17750,20250,17750,18300,372841,100840,137372811300,0.796373,1.000384,37171,11.22,0.59,1957,3.64,800,0.021326,SNT에너지,100840,중소형가치주
63,2022-04-29,27500,28500,27100,27900,46978,34590,122055636600,0.195433,0.998626,46433,31.08,0.60,901,4.46,1250,0.084053,인천도시가스,34590,중소형가치주
64,2022-04-29,44950,45500,44500,44800,918,4450,91628203779,0.280541,0.999451,78782,57.44,0.57,780,2.79,1250,0.078497,삼화왕관,4450,중소형가치주


In [120]:
style_index66_df['시가총액'][0]

39742282782000

In [125]:
capm =  [  0.78739859,   0.42978486,   2.00189077,   1.37835421,
         4.14944569,   1.55249615,   0.72400781,  -1.75086205,
         2.18403062,   5.70170908,  -2.33730102,  -0.15876842,
         2.7853228 ,  -6.26684356,   0.48044888,   2.38170313,
        -3.32426694,  -2.78876766,  -0.44047931,   0.5650682 ,
         0.18251952,  -6.27413352,   2.21503624,  -2.6341203 ,
         8.0174218 ,  -0.7395972 ,  -2.64201827,   3.35455055,
        -8.17252462,   6.56636813,  -5.13415618,  -3.2213663 ,
         6.43661007,   0.91926725,   0.37500652,   6.92071703,
        -3.80344047,  -6.56869312, -13.4738428 ,   9.57868461,
         3.69594451,   3.99764844,   5.91956538,   2.07092579,
         0.57758426,  -1.83150214,  11.29399537,  11.21528318,
         3.89820111,   1.3251964 ,   1.33042604,   1.87329807,
         1.38946051,   2.85349642,  -3.72587441,  -1.09972402,
        -5.0110892 ,  -3.58662794,  -4.66708287,   6.28103372,
       -10.1825828 ,   1.00816325,   1.13271602,  -2.97228895,
        -2.70867676]

In [126]:
sk = np.array(capm)
sk = sk/100
sk = sk + 1 

In [131]:
base = 1
for i in sk:
    base = base * i
(base-1)/5

0.03662834168608309

In [29]:
list66_df = pd.read_csv('./sample.csv')
list66_df

,Unnamed: 0,날짜,시가,고가,저가,종가,거래량,회사코드,시가총액,60beta,...,BPS,PER,PBR,EPS,DIV,DPS,기대수익률,Name,Symbol,분류기준
0,0,2022-04-29,187000,188000,184000,186000,767005,5380,39742282782000,1.202404,...,289609,9.85,0.65,18979,2.67,5000,0.423507,현대차,5380,대형가치주
1,1,2022-04-29,56800,57300,56700,57000,640368,17670,13020672034800,0.670489,...,53218,7.93,1.07,7191,11.68,6660,0.376312,SK텔레콤,17670,대형가치주
2,2,2022-04-29,72600,73400,72500,73300,252247,3550,14003834694000,0.943834,...,132574,4.76,0.56,15504,3.79,2800,0.178381,LG,3550,대형가치주
3,3,2022-04-29,33400,33500,33150,33450,83152,29780,3934255035200,0.402618,...,70408,6.47,0.47,5164,6.89,2300,-0.048669,삼성카드,29780,대형가치주
4,4,2022-04-29,121000,123500,121000,123000,27947,120,2794125567500,0.377914,...,178766,66.54,0.69,1841,0.00,0,-0.115830,CJ대한통운,120,대형가치주
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,61,2022-04-29,13600,13750,13550,13700,13209,1750,261602505000,0.978901,...,32750,2.20,0.42,6206,6.59,900,0.973198,한양증권,1750,중소형가치주
62,62,2022-04-29,17750,20250,17750,18300,372841,100840,137372811300,0.796373,...,37171,11.22,0.59,1957,3.64,800,0.021326,SNT에너지,100840,중소형가치주
63,63,2022-04-29,27500,28500,27100,27900,46978,34590,122055636600,0.195433,...,46433,31.08,0.60,901,4.46,1250,0.084053,인천도시가스,34590,중소형가치주
64,64,2022-04-29,44950,45500,44500,44800,918,4450,91628203779,0.280541,...,78782,57.44,0.57,780,2.79,1250,0.078497,삼화왕관,4450,중소형가치주


In [2]:
finnn = pd.read_csv('C:/Users/kht91/Desktop/유비온/프로젝트#1/INVEST IN BEST/startbootstrap-sb-admin-2-gh-pages/modules/datasets/중소형가치.csv')
finnn

,Unnamed: 0,종목명,종목코드,opt1,opt2,opt3
0,0,한미반도체,42700,25.79,12.78,2.66
1,1,NHN,181710,0.00,10.22,1.22
2,2,이노션,214320,0.00,9.20,9.19
3,3,한화투자증권,3530,0.00,8.05,8.05
4,4,DI동일,1530,31.33,5.70,5.82
5,5,NICE,34310,0.00,5.60,5.60
6,6,DL건설,1880,0.00,4.86,4.85
7,7,코오롱글로벌,3070,5.74,4.49,3.22
8,8,국도화학,7690,0.00,4.30,4.30
9,9,서흥,8490,0.00,3.84,9.70


In [81]:
list = []
for i in range(len(finnn)):
    list.append(int(list66_df[list66_df['Name'] == finnn['Unnamed: 0'][i]]['회사코드']))
# 거래소코드 6자리로 만들어주기
aa = list
bb = []
for i in range(len(aa)):
    bb.append(format(aa[i], '06'))

In [87]:
(finnn['opt1']*100).round(2).values

array([0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 9.700e-01,
       0.000e+00, 0.000e+00, 0.000e+00, 3.310e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 2.110e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 5.300e-01, 0.000e+00,
       0.000e+00, 6.060e+00, 0.000e+00, 5.900e-01, 0.000e+00, 1.000e-02,
       0.000e+00, 0.000e+00, 3.540e+00, 3.889e+01, 9.070e+00, 0.000e+00,
       1.597e+01, 0.000e+00, 0.000e+00, 0.000e+00, 5.000e-02, 2.630e+00,
       4.100e-01, 0.000e+00, 2.350e+00, 0.000e+00, 3.690e+00, 0.000e+00,
       8.460e+00, 1.230e+00, 1.000e-01, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00])

In [82]:
import pandas as pd
from pykrx import stock

name = []
list = bb
for code in list:
    c =stock.get_market_ticker_name(code)
    name.append(c)
    

big_g = pd.DataFrame({'종목명':name,
                      '종목코드': list,
                      'opt1':(finnn['opt1']*100).round(2).values,
                      'opt2':(finnn['opt2']*100).round(2).values,
                      'opt3':(finnn['opt3']*100).round(2).values})

big_g

,종목명,종목코드,opt1,opt2,opt3
0,현대차,005380,0.0,12.72,4.72
1,SK텔레콤,017670,0.0,4.19,4.24
2,LG,003550,0.0,3.78,3.88
3,삼성카드,029780,0.0,1.26,8.71
4,CJ대한통운,000120,0.0,0.98,0.98
...,...,...,...,...,...
57,한솔로지스틱스,009180,0.0,0.03,0.00
58,삼영화학,003720,0.0,0.04,0.00
59,진양산업,003780,0.0,0.03,0.00
60,덕양산업,024900,0.0,0.04,0.00


In [83]:
big_g.to_csv('./all.csv')

In [88]:
np.array(big_g['opt3'])

array([ 4.72,  4.24,  3.88,  8.71,  0.98,  0.  ,  0.  ,  0.27,  0.  ,
        1.54,  0.  ,  0.  ,  0.  ,  0.  ,  6.67,  0.  ,  0.  ,  0.  ,
        0.  ,  0.22,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  , 24.45, 12.62,  6.14, 11.21,  0.  ,  1.54,  2.69,
        9.65,  0.  ,  0.  ,  0.  ,  0.27,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  ,  0.14,  0.  ,  0.03,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ])

In [89]:
color =['#4e73df',
'#6610f2',
'#6f42c1',
'#e83e8c',
'#e74a3b',
'#fd7e14',
'#f6c23e',
'#1cc88a',
'#20c9a6',
'#36b9cc',
'#fff',
'#858796',
'#5a5c69',
'#4e73df',
'#858796',
'#1cc88a',
'#36b9cc',
'#f6c23e',
'#e74a3b',
'#f8f9fc',
'#5a5c69',
'#4e73df',
'#6610f2',
'#6f42c1',
'#e83e8c',
'#e74a3b',
'#fd7e14',
'#f6c23e',
'#1cc88a',
'#20c9a6',
'#36b9cc',
'#fff',
'#858796',
'#5a5c69',
'#4e73df',
'#858796',
'#1cc88a',
'#36b9cc',
'#f6c23e',
'#e74a3b',
'#f8f9fc',
'#5a5c69',
'#4e73df',
'#6610f2',
'#6f42c1',
'#e83e8c',
'#e74a3b',
'#fd7e14',
'#f6c23e',
'#1cc88a',
'#20c9a6',
'#36b9cc',
'#fff',
'#858796',
'#5a5c69',
'#4e73df',
'#858796',
'#1cc88a',
'#36b9cc',
'#f6c23e',
'#e74a3b',
'#f8f9fc',
'#5a5c69',
'#4e73df',
'#6610f2',
'#6f42c1',
'#e83e8c',
'#e74a3b',
'#fd7e14',
'#f6c23e',
'#1cc88a',
'#20c9a6',
'#36b9cc',
'#fff',
'#858796',
'#5a5c69',
'#4e73df',
'#858796',
'#1cc88a',
'#36b9cc',
'#f6c23e',
'#e74a3b',
'#f8f9fc',
'#5a5c69']

In [90]:
color[:len(big_g)]

['#4e73df',
 '#6610f2',
 '#6f42c1',
 '#e83e8c',
 '#e74a3b',
 '#fd7e14',
 '#f6c23e',
 '#1cc88a',
 '#20c9a6',
 '#36b9cc',
 '#fff',
 '#858796',
 '#5a5c69',
 '#4e73df',
 '#858796',
 '#1cc88a',
 '#36b9cc',
 '#f6c23e',
 '#e74a3b',
 '#f8f9fc',
 '#5a5c69',
 '#4e73df',
 '#6610f2',
 '#6f42c1',
 '#e83e8c',
 '#e74a3b',
 '#fd7e14',
 '#f6c23e',
 '#1cc88a',
 '#20c9a6',
 '#36b9cc',
 '#fff',
 '#858796',
 '#5a5c69',
 '#4e73df',
 '#858796',
 '#1cc88a',
 '#36b9cc',
 '#f6c23e',
 '#e74a3b',
 '#f8f9fc',
 '#5a5c69',
 '#4e73df',
 '#6610f2',
 '#6f42c1',
 '#e83e8c',
 '#e74a3b',
 '#fd7e14',
 '#f6c23e',
 '#1cc88a',
 '#20c9a6',
 '#36b9cc',
 '#fff',
 '#858796',
 '#5a5c69',
 '#4e73df',
 '#858796',
 '#1cc88a',
 '#36b9cc',
 '#f6c23e',
 '#e74a3b',
 '#f8f9fc']

In [ ]:
['#4e73df',
'#6610f2',
'#6f42c1',
'#e83e8c',
'#e74a3b',
'#fd7e14',
'#f6c23e',
'#1cc88a',
'#20c9a6',
'#36b9cc',
'#fff',
'#858796',
'#5a5c69',
'#4e73df',
'#858796',
'#1cc88a',
'#36b9cc',
'#f6c23e',
'#e74a3b',
'#f8f9fc',
'#5a5c69',
'#4e73df',
'#6610f2',
'#6f42c1']